In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

import os


In [2]:
def get_dat_dir_path()->str:
    return os.path.abspath('../dat')

def get_train_csv_path()->str:
    dat_dir = get_dat_dir_path()
    # print(dat_dir)
    return os.path.join(dat_dir, "train.csv")

def get_test_csv_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "test_x.csv")

def get_sample_sub_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "sample_submission.csv")


In [3]:
train = pd.read_csv(get_train_csv_path())
test = pd.read_csv(get_test_csv_path())
sample_submission = pd.read_csv(get_sample_sub_path())

In [4]:
# preprocesscing

In [5]:
def alpha_num(txt:str)->str:
    return re.sub(r"[^A-Za-z0-9 ]", "", txt)

train["text"] = train["text"].apply(alpha_num)

In [6]:
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [7]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

def remove_stopwords(text:str) -> str:
    final_txt = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_txt.append(i.strip())
    return " ".join(final_txt)

In [8]:
train["text"] = train["text"].apply(alpha_num).apply(remove_stopwords)
test["text"] = test["text"].apply(alpha_num).apply(remove_stopwords)

In [9]:
train

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing Janes last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands Dont write an...,3
...,...,...,...
54874,54874,Mr Smith odin whispered hardly dared hope come,2
54875,54875,told plan captain us settled details accomplis...,4
54876,54876,sincere wellwisher friend sister LUCY odin,1
54877,54877,wanted lend money,3


In [10]:
# x_train = train["text"].values

In [11]:
# x_train

In [12]:
x_train = np.array([x for x in train["text"]])
x_test = np.array([x for x in test["text"]])
y_train = np.array([x for x in train["author"]])

In [13]:
# x_train

In [14]:
# Modeling

In [15]:
vocab_size = 20000
embedding_dim = 100
max_length = 500
padding_type = "post"

In [16]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [17]:
list(word_index)[:10]

['odin', 'not', 'said', 'no', 'one', 'mr', 'will', 'upon', 'now', 'man']

In [18]:
train_sequence = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequence, padding=padding_type, maxlen=max_length)

test_sequence = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequence, padding=padding_type, maxlen=max_length)

In [19]:
train_padded

array([[  141,  7259,    20, ...,     0,     0,     0],
       [  217,    58,   221, ...,     0,     0,     0],
       [  682,     5,    59, ...,     0,     0,     0],
       ...,
       [ 2407, 19738,   126, ...,     0,     0,     0],
       [  316,  3532,   164, ...,     0,     0,     0],
       [  203,     2,   811, ...,     0,     0,     0]])

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(124, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])

In [21]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
print(model.summary())            

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 100)          2000000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 124)               12524     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 625       
Total params: 2,013,149
Trainable params: 2,013,149
Non-trainable params: 0
_________________________________________________________________
None


In [114]:
num_epochs = 20
history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2, validation_split=.2)

Epoch 1/20
1372/1372 - 8s - loss: 1.5590 - accuracy: 0.2848 - val_loss: 1.4989 - val_accuracy: 0.3237
Epoch 2/20
1372/1372 - 8s - loss: 1.2886 - accuracy: 0.4550 - val_loss: 1.1787 - val_accuracy: 0.4989
Epoch 3/20
1372/1372 - 8s - loss: 1.0910 - accuracy: 0.5430 - val_loss: 1.0772 - val_accuracy: 0.5492
Epoch 4/20
1372/1372 - 10s - loss: 0.9843 - accuracy: 0.6039 - val_loss: 1.0045 - val_accuracy: 0.5983
Epoch 5/20
1372/1372 - 7s - loss: 0.8727 - accuracy: 0.6687 - val_loss: 0.9318 - val_accuracy: 0.6358
Epoch 6/20
1372/1372 - 8s - loss: 0.7706 - accuracy: 0.7161 - val_loss: 0.8550 - val_accuracy: 0.6800
Epoch 7/20
1372/1372 - 8s - loss: 0.6996 - accuracy: 0.7446 - val_loss: 0.8313 - val_accuracy: 0.6911
Epoch 8/20
1372/1372 - 8s - loss: 0.6453 - accuracy: 0.7648 - val_loss: 0.7959 - val_accuracy: 0.7042
Epoch 9/20
1372/1372 - 8s - loss: 0.6046 - accuracy: 0.7783 - val_loss: 0.8156 - val_accuracy: 0.7013
Epoch 10/20
1372/1372 - 9s - loss: 0.5686 - accuracy: 0.7931 - val_loss: 0.7813 -

In [115]:
pred = model.predict_proba(test_padded)

In [116]:
pred

array([[2.23496645e-05, 1.03462838e-01, 4.24345285e-02, 8.32242906e-01,
        2.18374562e-02],
       [3.54147702e-02, 5.59135199e-01, 9.02234390e-02, 8.84561241e-02,
        2.26770446e-01],
       [9.99983072e-01, 1.56119022e-05, 1.14379066e-08, 1.10233024e-10,
        1.27408168e-06],
       ...,
       [8.16341781e-05, 9.99909282e-01, 8.04826428e-10, 1.95369262e-06,
        7.13737518e-06],
       [8.93631732e-05, 9.99848962e-01, 6.25036911e-09, 7.89445039e-07,
        6.07830916e-05],
       [9.99983191e-01, 4.34709727e-06, 4.26566839e-06, 1.91045029e-08,
        8.23619757e-06]], dtype=float32)

In [117]:
pred.shape

(19617, 5)

In [118]:
test_padded.shape

(19617, 500)

In [119]:
sample_submission[[str(i) for i in range(5)]] = pred
sample_submission

,index,0,1,2,3,4
0,0,2.234966e-05,1.034628e-01,4.243453e-02,8.322429e-01,2.183746e-02
1,1,3.541477e-02,5.591352e-01,9.022344e-02,8.845612e-02,2.267704e-01
2,2,9.999831e-01,1.561190e-05,1.143791e-08,1.102330e-10,1.274082e-06
3,3,8.926961e-07,1.792368e-12,9.995331e-01,1.846117e-09,4.661073e-04
4,4,9.257374e-01,1.335113e-04,1.355598e-03,7.277032e-02,3.078114e-06
...,...,...,...,...,...,...
19612,19612,1.330739e-07,9.999999e-01,8.891108e-21,3.169428e-11,3.078867e-14
19613,19613,2.404358e-04,1.468442e-05,4.297000e-06,5.498563e-15,9.997405e-01
19614,19614,8.163418e-05,9.999093e-01,8.048264e-10,1.953693e-06,7.137375e-06
19615,19615,8.936317e-05,9.998490e-01,6.250369e-09,7.894450e-07,6.078309e-05


In [120]:
sample_submission.to_csv("submission.csv", index=False, encoding="utf-8")